Includes:

Reading rasters and masking based on polygon coordinates

Creating bins (categories) for different statistics

Aggregating raster stats to regional level for each of the bins

Libraries

In [2]:
#Common libs
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import rasterio
import geopandas as gpd
from rasterstats import zonal_stats

In [4]:
#Visualise libs
import matplotlib.pylab as plt
import seaborn as sns

In [6]:
gdf = gpd.read_file("C:/Users/Dima/Desktop/MIP/stations/regions_selected_labeled.gpkg")

### Nitrogen

In [5]:
raster_path = "Soil_and_Landscape_Grid_National_Soil_Attribute_Maps_-_Total_Nitrogen_(3__resolution)_-_Release_2-HWRDfZ8e-/data/NTO_000_005_EV_N_P_AU_NAT_C_20231101.tif"

In [ ]:
with rasterio.open(raster_path) as src:
    band1 = src.read(1, masked = True)   #first band
    raster_min = band1.min()
    raster_max = band1.max()

print("Min:", raster_min, "Max:", raster_max)

Min: 0.02 Max: 0.7


In [ ]:
bins = np.linspace(raster_min, raster_max, 8) # so that enough edges for histograms
print("Bins:", bins)

Bins: [0.02       0.11714286 0.21428571 0.31142857 0.40857142 0.50571428
 0.60285713 0.69999999]


In [45]:
bin_labels = [f"bin_{i}" for i in range(len(bins)-1)]
bin_labels

['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'bin_5', 'bin_6']

In [ ]:
stats = zonal_stats(
    gdf,
    raster_path,
    categorical = True,
    all_touched = True,  # include partial overlaps
    raster_out = False
)

In [ ]:
def get_binned_hist(geom, raster, bins):
    mask = rasterio.features.geometry_mask([geom], transform = raster.transform,
                                           invert = True, out_shape = raster.shape)
    arr = raster.read(1, masked=True)
    arr = arr[mask]
    arr = arr.compressed()  # drop masked
    
    # Bin values
    counts, _ = np.histogram(arr, bins=bins)
    total = counts.sum()
    if total == 0:
        return [0]*len(counts)
    return (counts / total).tolist()

In [48]:
with rasterio.open(raster_path) as raster:    
    features = []
    for geom in gdf.geometry:
        feats = get_binned_hist(geom, raster, bins)
        features.append(feats)

    features_df = pd.DataFrame(features, columns=bin_labels)

In [49]:
features_df

,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6
0,0.628219,0.347783,0.021059,0.002435,0.000456,0.000047,0.000000
1,0.973983,0.023493,0.002314,0.000208,0.000002,0.000000,0.000000
2,0.742361,0.257634,0.000005,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.006054,0.699257,0.213501,0.040172,0.040554,0.000462
4,0.510316,0.487054,0.002530,0.000100,0.000000,0.000000,0.000000
5,0.406046,0.539823,0.054054,0.000076,0.000000,0.000000,0.000000
6,0.133807,0.675725,0.189032,0.001422,0.000014,0.000000,0.000000
7,0.242226,0.667477,0.088964,0.001324,0.000009,0.000000,0.000000
8,0.350251,0.593654,0.055366,0.000706,0.000017,0.000005,0.000001
9,0.518498,0.451991,0.029445,0.000064,0.000002,0.000000,0.000000


In [50]:
bins_gdf = gdf.reset_index(drop=True).join(features_df)

In [51]:
bins_gdf

,id,RegionName,geometry,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6
0,1,QLD Condamine-Balonne,"MULTIPOLYGON (((146.90174 -28.95673, 146.5 -28...",0.628219,0.347783,0.021059,0.002435,0.000456,0.000047,0.000000
1,2,NSW Barwon Darling,"MULTIPOLYGON (((144.18743 -29.03522, 148.2859 ...",0.973983,0.023493,0.002314,0.000208,0.000002,0.000000,0.000000
2,3,QLD Moonie,"MULTIPOLYGON (((148.60325 -28.94576, 148.77205...",0.742361,0.257634,0.000005,0.000000,0.000000,0.000000,0.000000
3,4,NSW Murray Above,"MULTIPOLYGON (((147.31699 -35.96701, 147.34906...",0.000000,0.006054,0.699257,0.213501,0.040172,0.040554,0.000462
4,5,NSW Lower Darling,"MULTIPOLYGON (((141.05399 -33.28225, 141.08446...",0.510316,0.487054,0.002530,0.000100,0.000000,0.000000,0.000000
5,6,QLD Border Rivers,"MULTIPOLYGON (((149.00561 -28.92837, 148.99289...",0.406046,0.539823,0.054054,0.000076,0.000000,0.000000,0.000000
6,7,NSW Border Rivers,"MULTIPOLYGON (((149.15254 -28.85159, 149.23521...",0.133807,0.675725,0.189032,0.001422,0.000014,0.000000,0.000000
7,8,NSW Gwydir,"MULTIPOLYGON (((151.68274 -29.95479, 151.68618...",0.242226,0.667477,0.088964,0.001324,0.000009,0.000000,0.000000
8,9,NSW Namoi,"MULTIPOLYGON (((148.33494 -29.93474, 148.42915...",0.350251,0.593654,0.055366,0.000706,0.000017,0.000005,0.000001
9,10,NSW Macquarie-Castlereagh,"MULTIPOLYGON (((146.52897 -32.31278, 146.4993 ...",0.518498,0.451991,0.029445,0.000064,0.000002,0.000000,0.000000


In [53]:
bins_gdf.to_csv("nitrogen_binned.csv")

### Phosphorus

In [54]:
raster_path = "Soil_and_Landscape_Grid_National_Soil_Attribute_Maps_-_Total_Phosphorus_(3__resolution)_-_Release_2-WIfTmxnD-/data/PTO_000_005_EV_N_P_AU_NAT_C_20231101.tif"

In [ ]:
with rasterio.open(raster_path) as src:
    band1 = src.read(1, masked=True)   # first band
    raster_min = band1.min()
    raster_max = band1.max()

print("Min:", raster_min, "Max:", raster_max)

Min: 0.01 Max: 0.64


In [56]:
bins = np.linspace(raster_min, raster_max, 8) # so that enough edges for histograms
print("Bins:", bins)

Bins: [0.01       0.1        0.19       0.27999999 0.36999999 0.45999999
 0.54999999 0.63999999]


In [57]:
bin_labels = [f"bin_{i}" for i in range(len(bins)-1)]
bin_labels

['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'bin_5', 'bin_6']

In [ ]:
stats = zonal_stats(
    gdf,
    raster_path,
    categorical = True,
    all_touched = True,
    raster_out = False
)

In [ ]:
with rasterio.open(raster_path) as raster:    
    features = []
    for geom in gdf.geometry:
        feats = get_binned_hist(geom, raster, bins)
        features.append(feats)

    features_df = pd.DataFrame(features, columns = bin_labels)

In [60]:
features_df

,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6
0,0.933798,0.063448,2.478503e-03,0.000247,2.851467e-05,0.000000,0.000000
1,0.998382,0.001485,1.274587e-04,0.000005,9.306953e-08,0.000000,0.000000
2,0.999994,0.000006,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
3,0.000190,0.088860,3.516639e-01,0.415737,1.052755e-01,0.037934,0.000340
4,0.909281,0.090719,8.133159e-07,0.000000,0.000000e+00,0.000000,0.000000
5,0.959824,0.038245,1.727408e-03,0.000198,4.581984e-06,0.000000,0.000000
6,0.678183,0.306903,1.446858e-02,0.000443,2.077088e-06,0.000000,0.000000
7,0.683479,0.303165,1.320707e-02,0.000148,3.244901e-07,0.000000,0.000000
8,0.758784,0.212159,2.533737e-02,0.003639,8.130264e-05,0.000000,0.000000
9,0.837215,0.153014,8.491219e-03,0.001271,8.903545e-06,0.000000,0.000000


In [ ]:
bins_gdf = gdf.reset_index(drop = True).join(features_df)

In [62]:
bins_gdf

,id,RegionName,geometry,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6
0,1,QLD Condamine-Balonne,"MULTIPOLYGON (((146.90174 -28.95673, 146.5 -28...",0.933798,0.063448,2.478503e-03,0.000247,2.851467e-05,0.000000,0.000000
1,2,NSW Barwon Darling,"MULTIPOLYGON (((144.18743 -29.03522, 148.2859 ...",0.998382,0.001485,1.274587e-04,0.000005,9.306953e-08,0.000000,0.000000
2,3,QLD Moonie,"MULTIPOLYGON (((148.60325 -28.94576, 148.77205...",0.999994,0.000006,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
3,4,NSW Murray Above,"MULTIPOLYGON (((147.31699 -35.96701, 147.34906...",0.000190,0.088860,3.516639e-01,0.415737,1.052755e-01,0.037934,0.000340
4,5,NSW Lower Darling,"MULTIPOLYGON (((141.05399 -33.28225, 141.08446...",0.909281,0.090719,8.133159e-07,0.000000,0.000000e+00,0.000000,0.000000
5,6,QLD Border Rivers,"MULTIPOLYGON (((149.00561 -28.92837, 148.99289...",0.959824,0.038245,1.727408e-03,0.000198,4.581984e-06,0.000000,0.000000
6,7,NSW Border Rivers,"MULTIPOLYGON (((149.15254 -28.85159, 149.23521...",0.678183,0.306903,1.446858e-02,0.000443,2.077088e-06,0.000000,0.000000
7,8,NSW Gwydir,"MULTIPOLYGON (((151.68274 -29.95479, 151.68618...",0.683479,0.303165,1.320707e-02,0.000148,3.244901e-07,0.000000,0.000000
8,9,NSW Namoi,"MULTIPOLYGON (((148.33494 -29.93474, 148.42915...",0.758784,0.212159,2.533737e-02,0.003639,8.130264e-05,0.000000,0.000000
9,10,NSW Macquarie-Castlereagh,"MULTIPOLYGON (((146.52897 -32.31278, 146.4993 ...",0.837215,0.153014,8.491219e-03,0.001271,8.903545e-06,0.000000,0.000000


In [ ]:
bins_gdf.to_csv("phosphorus_binned.csv")

### pH

In [64]:
raster_path = "Soil_and_Landscape_Grid_National_Soil_Attribute_Maps_-_pH_-_CaCl2_(3__resolution)_-_Release_2-Qx6DCvCL-/data/pHc_000_005_EV_N_P_AU_NAT_C_20210913.tif"

In [ ]:
with rasterio.open(raster_path) as src:
    band1 = src.read(1, masked = True)
    raster_min = band1.min()
    raster_max = band1.max()

print("Min:", raster_min, "Max:", raster_max)

Min: 3.42 Max: 8.14


In [ ]:
bins = np.linspace(raster_min, raster_max, 6)
print("Bins:", bins)

Bins: [3.42000008 4.36400013 5.30800018 6.25200024 7.19600029 8.14000034]


In [ ]:
bin_labels = [f"bin_{i}" for i in range(len(bins) - 1)]
bin_labels

['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']

In [ ]:
stats = zonal_stats(
    gdf,
    raster_path,
    categorical = True,
    all_touched = True,
    raster_out = False
)

In [69]:
with rasterio.open(raster_path) as raster:    
    features = []
    for geom in gdf.geometry:
        feats = get_binned_hist(geom, raster, bins)
        features.append(feats)

    features_df = pd.DataFrame(features, columns=bin_labels)

In [70]:
features_df

,bin_0,bin_1,bin_2,bin_3,bin_4
0,0.000009,0.304620,0.552931,0.140210,0.002229
1,0.000000,0.091398,0.515253,0.354172,0.039177
2,0.000000,0.181288,0.742814,0.075898,0.000000
3,0.003034,0.890713,0.106253,0.000000,0.000000
4,0.000000,0.000000,0.103675,0.866637,0.029689
5,0.000000,0.288485,0.593593,0.117810,0.000113
6,0.000425,0.512433,0.358522,0.128547,0.000072
7,0.000000,0.202680,0.449123,0.348197,0.000000
8,0.000000,0.241491,0.527929,0.229911,0.000669
9,0.000703,0.435871,0.472666,0.090229,0.000531


In [71]:
bins_gdf = gdf.reset_index(drop=True).join(features_df)

In [72]:
bins_gdf.drop('id', axis = 1, inplace = True)
bins_gdf

,RegionName,geometry,bin_0,bin_1,bin_2,bin_3,bin_4
0,QLD Condamine-Balonne,"MULTIPOLYGON (((146.90174 -28.95673, 146.5 -28...",0.000009,0.304620,0.552931,0.140210,0.002229
1,NSW Barwon Darling,"MULTIPOLYGON (((144.18743 -29.03522, 148.2859 ...",0.000000,0.091398,0.515253,0.354172,0.039177
2,QLD Moonie,"MULTIPOLYGON (((148.60325 -28.94576, 148.77205...",0.000000,0.181288,0.742814,0.075898,0.000000
3,NSW Murray Above,"MULTIPOLYGON (((147.31699 -35.96701, 147.34906...",0.003034,0.890713,0.106253,0.000000,0.000000
4,NSW Lower Darling,"MULTIPOLYGON (((141.05399 -33.28225, 141.08446...",0.000000,0.000000,0.103675,0.866637,0.029689
5,QLD Border Rivers,"MULTIPOLYGON (((149.00561 -28.92837, 148.99289...",0.000000,0.288485,0.593593,0.117810,0.000113
6,NSW Border Rivers,"MULTIPOLYGON (((149.15254 -28.85159, 149.23521...",0.000425,0.512433,0.358522,0.128547,0.000072
7,NSW Gwydir,"MULTIPOLYGON (((151.68274 -29.95479, 151.68618...",0.000000,0.202680,0.449123,0.348197,0.000000
8,NSW Namoi,"MULTIPOLYGON (((148.33494 -29.93474, 148.42915...",0.000000,0.241491,0.527929,0.229911,0.000669
9,NSW Macquarie-Castlereagh,"MULTIPOLYGON (((146.52897 -32.31278, 146.4993 ...",0.000703,0.435871,0.472666,0.090229,0.000531


In [73]:
bins_gdf.to_csv("ph_binned.csv")

### Soil Type

In [7]:
raster_path = "Soil_and_Landscape_Grid_National_Soil_Attribute_Maps_-_Australian_Soil_Classification_Map_(3__resolution)_-_Release_1-3gj4oPwN-/data/ASC_EV_C_P_AU_TRN_N.cog.tif"

In [8]:
stats = zonal_stats(
    gdf,
    raster_path,
    categorical = True,
    all_touched = True,
    raster_out = False
)

In [13]:
seen_types = set()
for stat in stats:
    seen_types.update(stat.keys())
missing_types = set(range(1, 15)) - set(seen_types)
print("Missing soil type codes:", missing_types)

Missing soil type codes: {14}


In [9]:
soi_types = list(range(1, 15))
soi_types

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
cat_labels = [f"type_{i}" for i in range(1, 14)]
cat_labels

['type_1',
 'type_2',
 'type_3',
 'type_4',
 'type_5',
 'type_6',
 'type_7',
 'type_8',
 'type_9',
 'type_10',
 'type_11',
 'type_12',
 'type_13']

In [ ]:
features = []
for stat in stats:
    total_count = sum(stat.values())
    pct_features = {category: (count / total_count * 100) for category, count in stat.items()}
    features.append(pct_features)

# Create df
features_df = pd.DataFrame(features).fillna(0)

# Rename columns
features_df.columns = cat_labels

In [16]:
features_df

,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,type_9,type_10,type_11,type_12,type_13
0,35.347101,22.049089,5.679765,0.622659,0.327819,0.003821,3.305808,32.576442,0.005787,0.074742,0.006968,0.000000,0.000000
1,34.235789,1.399530,0.468904,3.173449,0.001455,0.000000,1.103540,59.358891,0.000000,0.257142,0.001302,0.000000,0.000000
2,35.044889,29.262590,8.624445,0.094318,0.000000,0.000000,0.213891,26.759682,0.000000,0.000184,0.000000,0.000000,0.000000
3,0.000000,0.012851,42.943507,19.679128,0.196253,0.438036,19.763578,16.837402,0.002937,0.123370,0.000000,0.002937,0.000000
4,12.043621,1.056868,3.518442,17.872968,0.000000,0.000000,2.165538,11.832438,0.000000,0.102052,51.408074,0.000000,0.000000
5,30.217194,52.871211,3.192016,2.523157,0.000000,0.516122,6.598406,2.782747,0.000844,1.298304,0.000000,0.000000,0.000000
6,34.260765,18.686251,8.842331,7.333070,0.244514,1.277341,16.864656,6.538957,0.003149,5.948925,0.000000,0.000041,0.000000
7,55.717780,7.060628,6.417593,14.617804,0.573288,0.565064,12.069528,1.487927,0.000000,1.490259,0.000000,0.000130,0.000000
8,41.185256,17.002504,5.398950,21.342490,0.116707,0.188132,13.574718,0.415647,0.000078,0.774040,0.001129,0.000000,0.000350
9,26.708438,1.629975,3.624785,45.717839,0.187251,0.136892,9.765206,11.805346,0.000255,0.423011,0.001001,0.000000,0.000000


In [28]:
features_df = features_df / 100

In [29]:
bins_gdf = gdf.reset_index(drop=True).join(features_df)

In [30]:
bins_gdf.drop('id', axis = 1, inplace = True)
bins_gdf

,RegionName,geometry,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,type_9,type_10,type_11,type_12,type_13
0,QLD Condamine-Balonne,"MULTIPOLYGON (((146.90174 -28.95673, 146.5 -28...",0.353471,0.220491,0.056798,0.006227,0.003278,0.000038,0.033058,0.325764,5.787290e-05,0.000747,0.000070,0.000000e+00,0.000000
1,NSW Barwon Darling,"MULTIPOLYGON (((144.18743 -29.03522, 148.2859 ...",0.342358,0.013995,0.004689,0.031734,0.000015,0.000000,0.011035,0.593589,0.000000e+00,0.002571,0.000013,0.000000e+00,0.000000
2,QLD Moonie,"MULTIPOLYGON (((148.60325 -28.94576, 148.77205...",0.350449,0.292626,0.086244,0.000943,0.000000,0.000000,0.002139,0.267597,0.000000e+00,0.000002,0.000000,0.000000e+00,0.000000
3,NSW Murray Above,"MULTIPOLYGON (((147.31699 -35.96701, 147.34906...",0.000000,0.000129,0.429435,0.196791,0.001963,0.004380,0.197636,0.168374,2.937375e-05,0.001234,0.000000,2.937375e-05,0.000000
4,NSW Lower Darling,"MULTIPOLYGON (((141.05399 -33.28225, 141.08446...",0.120436,0.010569,0.035184,0.178730,0.000000,0.000000,0.021655,0.118324,0.000000e+00,0.001021,0.514081,0.000000e+00,0.000000
5,QLD Border Rivers,"MULTIPOLYGON (((149.00561 -28.92837, 148.99289...",0.302172,0.528712,0.031920,0.025232,0.000000,0.005161,0.065984,0.027827,8.438534e-06,0.012983,0.000000,0.000000e+00,0.000000
6,NSW Border Rivers,"MULTIPOLYGON (((149.15254 -28.85159, 149.23521...",0.342608,0.186863,0.088423,0.073331,0.002445,0.012773,0.168647,0.065390,3.148609e-05,0.059489,0.000000,4.142907e-07,0.000000
7,NSW Gwydir,"MULTIPOLYGON (((151.68274 -29.95479, 151.68618...",0.557178,0.070606,0.064176,0.146178,0.005733,0.005651,0.120695,0.014879,0.000000e+00,0.014903,0.000000,1.295201e-06,0.000000
8,NSW Namoi,"MULTIPOLYGON (((148.33494 -29.93474, 148.42915...",0.411853,0.170025,0.053989,0.213425,0.001167,0.001881,0.135747,0.004156,7.786928e-07,0.007740,0.000011,0.000000e+00,0.000004
9,NSW Macquarie-Castlereagh,"MULTIPOLYGON (((146.52897 -32.31278, 146.4993 ...",0.267084,0.016300,0.036248,0.457178,0.001873,0.001369,0.097652,0.118053,2.554723e-06,0.004230,0.000010,0.000000e+00,0.000000


In [32]:
bins_gdf.to_csv("soil_types_categories.csv", index = False)

### Combining

In [35]:
nitrogen = pd.read_csv("nitrogen_binned.csv").drop(["Unnamed: 0", "id"], axis = 1)
nitrogen

,RegionName,geometry,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6
0,QLD Condamine-Balonne,MULTIPOLYGON (((146.90174272390513 -28.9567308...,0.628219,0.347783,0.021059,0.002435,0.000456,0.000047,0.000000
1,NSW Barwon Darling,MULTIPOLYGON (((144.1874344326267 -29.03522301...,0.973983,0.023493,0.002314,0.000208,0.000002,0.000000,0.000000
2,QLD Moonie,MULTIPOLYGON (((148.6032494139603 -28.94575887...,0.742361,0.257634,0.000005,0.000000,0.000000,0.000000,0.000000
3,NSW Murray Above,MULTIPOLYGON (((147.31699138040665 -35.9670059...,0.000000,0.006054,0.699257,0.213501,0.040172,0.040554,0.000462
4,NSW Lower Darling,MULTIPOLYGON (((141.05399239322216 -33.2822541...,0.510316,0.487054,0.002530,0.000100,0.000000,0.000000,0.000000
5,QLD Border Rivers,MULTIPOLYGON (((149.00561064400677 -28.9283685...,0.406046,0.539823,0.054054,0.000076,0.000000,0.000000,0.000000
6,NSW Border Rivers,MULTIPOLYGON (((149.15253736606823 -28.8515927...,0.133807,0.675725,0.189032,0.001422,0.000014,0.000000,0.000000
7,NSW Gwydir,MULTIPOLYGON (((151.682735563879 -29.954786754...,0.242226,0.667477,0.088964,0.001324,0.000009,0.000000,0.000000
8,NSW Namoi,MULTIPOLYGON (((148.33494353037074 -29.9347375...,0.350251,0.593654,0.055366,0.000706,0.000017,0.000005,0.000001
9,NSW Macquarie-Castlereagh,MULTIPOLYGON (((146.52897465087446 -32.3127849...,0.518498,0.451991,0.029445,0.000064,0.000002,0.000000,0.000000


In [36]:
nitrogen.rename(columns = {"bin_0" : "n_bin_0", "bin_1" : "n_bin_1", "bin_2" : "n_bin_2", "bin_3" : "n_bin_3", "bin_4" : "n_bin_4", "bin_5" : "n_bin_5", "bin_6" : "n_bin_6"}, inplace = True)
nitrogen

,RegionName,geometry,n_bin_0,n_bin_1,n_bin_2,n_bin_3,n_bin_4,n_bin_5,n_bin_6
0,QLD Condamine-Balonne,MULTIPOLYGON (((146.90174272390513 -28.9567308...,0.628219,0.347783,0.021059,0.002435,0.000456,0.000047,0.000000
1,NSW Barwon Darling,MULTIPOLYGON (((144.1874344326267 -29.03522301...,0.973983,0.023493,0.002314,0.000208,0.000002,0.000000,0.000000
2,QLD Moonie,MULTIPOLYGON (((148.6032494139603 -28.94575887...,0.742361,0.257634,0.000005,0.000000,0.000000,0.000000,0.000000
3,NSW Murray Above,MULTIPOLYGON (((147.31699138040665 -35.9670059...,0.000000,0.006054,0.699257,0.213501,0.040172,0.040554,0.000462
4,NSW Lower Darling,MULTIPOLYGON (((141.05399239322216 -33.2822541...,0.510316,0.487054,0.002530,0.000100,0.000000,0.000000,0.000000
5,QLD Border Rivers,MULTIPOLYGON (((149.00561064400677 -28.9283685...,0.406046,0.539823,0.054054,0.000076,0.000000,0.000000,0.000000
6,NSW Border Rivers,MULTIPOLYGON (((149.15253736606823 -28.8515927...,0.133807,0.675725,0.189032,0.001422,0.000014,0.000000,0.000000
7,NSW Gwydir,MULTIPOLYGON (((151.682735563879 -29.954786754...,0.242226,0.667477,0.088964,0.001324,0.000009,0.000000,0.000000
8,NSW Namoi,MULTIPOLYGON (((148.33494353037074 -29.9347375...,0.350251,0.593654,0.055366,0.000706,0.000017,0.000005,0.000001
9,NSW Macquarie-Castlereagh,MULTIPOLYGON (((146.52897465087446 -32.3127849...,0.518498,0.451991,0.029445,0.000064,0.000002,0.000000,0.000000


In [37]:
phosphorus = pd.read_csv("phosphorus_binned.csv").drop(["Unnamed: 0", "id"], axis = 1)
phosphorus

,RegionName,geometry,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6
0,QLD Condamine-Balonne,MULTIPOLYGON (((146.90174272390513 -28.9567308...,0.933798,0.063448,2.478503e-03,0.000247,2.851467e-05,0.000000,0.000000
1,NSW Barwon Darling,MULTIPOLYGON (((144.1874344326267 -29.03522301...,0.998382,0.001485,1.274587e-04,0.000005,9.306953e-08,0.000000,0.000000
2,QLD Moonie,MULTIPOLYGON (((148.6032494139603 -28.94575887...,0.999994,0.000006,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
3,NSW Murray Above,MULTIPOLYGON (((147.31699138040665 -35.9670059...,0.000190,0.088860,3.516639e-01,0.415737,1.052755e-01,0.037934,0.000340
4,NSW Lower Darling,MULTIPOLYGON (((141.05399239322216 -33.2822541...,0.909281,0.090719,8.133159e-07,0.000000,0.000000e+00,0.000000,0.000000
5,QLD Border Rivers,MULTIPOLYGON (((149.00561064400677 -28.9283685...,0.959824,0.038245,1.727408e-03,0.000198,4.581984e-06,0.000000,0.000000
6,NSW Border Rivers,MULTIPOLYGON (((149.15253736606823 -28.8515927...,0.678183,0.306903,1.446858e-02,0.000443,2.077088e-06,0.000000,0.000000
7,NSW Gwydir,MULTIPOLYGON (((151.682735563879 -29.954786754...,0.683479,0.303165,1.320707e-02,0.000148,3.244901e-07,0.000000,0.000000
8,NSW Namoi,MULTIPOLYGON (((148.33494353037074 -29.9347375...,0.758784,0.212159,2.533737e-02,0.003639,8.130264e-05,0.000000,0.000000
9,NSW Macquarie-Castlereagh,MULTIPOLYGON (((146.52897465087446 -32.3127849...,0.837215,0.153014,8.491219e-03,0.001271,8.903545e-06,0.000000,0.000000


In [38]:
phosphorus.rename(columns = {"bin_0" : "p_bin_0", "bin_1" : "p_bin_1", "bin_2" : "p_bin_2", "bin_3" : "p_bin_3", "bin_4" : "p_bin_4", "bin_5" : "p_bin_5", "bin_6" : "p_bin_6"}, inplace = True)
phosphorus

,RegionName,geometry,p_bin_0,p_bin_1,p_bin_2,p_bin_3,p_bin_4,p_bin_5,p_bin_6
0,QLD Condamine-Balonne,MULTIPOLYGON (((146.90174272390513 -28.9567308...,0.933798,0.063448,2.478503e-03,0.000247,2.851467e-05,0.000000,0.000000
1,NSW Barwon Darling,MULTIPOLYGON (((144.1874344326267 -29.03522301...,0.998382,0.001485,1.274587e-04,0.000005,9.306953e-08,0.000000,0.000000
2,QLD Moonie,MULTIPOLYGON (((148.6032494139603 -28.94575887...,0.999994,0.000006,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
3,NSW Murray Above,MULTIPOLYGON (((147.31699138040665 -35.9670059...,0.000190,0.088860,3.516639e-01,0.415737,1.052755e-01,0.037934,0.000340
4,NSW Lower Darling,MULTIPOLYGON (((141.05399239322216 -33.2822541...,0.909281,0.090719,8.133159e-07,0.000000,0.000000e+00,0.000000,0.000000
5,QLD Border Rivers,MULTIPOLYGON (((149.00561064400677 -28.9283685...,0.959824,0.038245,1.727408e-03,0.000198,4.581984e-06,0.000000,0.000000
6,NSW Border Rivers,MULTIPOLYGON (((149.15253736606823 -28.8515927...,0.678183,0.306903,1.446858e-02,0.000443,2.077088e-06,0.000000,0.000000
7,NSW Gwydir,MULTIPOLYGON (((151.682735563879 -29.954786754...,0.683479,0.303165,1.320707e-02,0.000148,3.244901e-07,0.000000,0.000000
8,NSW Namoi,MULTIPOLYGON (((148.33494353037074 -29.9347375...,0.758784,0.212159,2.533737e-02,0.003639,8.130264e-05,0.000000,0.000000
9,NSW Macquarie-Castlereagh,MULTIPOLYGON (((146.52897465087446 -32.3127849...,0.837215,0.153014,8.491219e-03,0.001271,8.903545e-06,0.000000,0.000000


In [39]:
ph = pd.read_csv("ph_binned.csv").drop(["Unnamed: 0"], axis = 1)
ph

,RegionName,geometry,bin_0,bin_1,bin_2,bin_3,bin_4
0,QLD Condamine-Balonne,MULTIPOLYGON (((146.90174272390513 -28.9567308...,0.000009,0.304620,0.552931,0.140210,0.002229
1,NSW Barwon Darling,MULTIPOLYGON (((144.1874344326267 -29.03522301...,0.000000,0.091398,0.515253,0.354172,0.039177
2,QLD Moonie,MULTIPOLYGON (((148.6032494139603 -28.94575887...,0.000000,0.181288,0.742814,0.075898,0.000000
3,NSW Murray Above,MULTIPOLYGON (((147.31699138040665 -35.9670059...,0.003034,0.890713,0.106253,0.000000,0.000000
4,NSW Lower Darling,MULTIPOLYGON (((141.05399239322216 -33.2822541...,0.000000,0.000000,0.103675,0.866637,0.029689
5,QLD Border Rivers,MULTIPOLYGON (((149.00561064400677 -28.9283685...,0.000000,0.288485,0.593593,0.117810,0.000113
6,NSW Border Rivers,MULTIPOLYGON (((149.15253736606823 -28.8515927...,0.000425,0.512433,0.358522,0.128547,0.000072
7,NSW Gwydir,MULTIPOLYGON (((151.682735563879 -29.954786754...,0.000000,0.202680,0.449123,0.348197,0.000000
8,NSW Namoi,MULTIPOLYGON (((148.33494353037074 -29.9347375...,0.000000,0.241491,0.527929,0.229911,0.000669
9,NSW Macquarie-Castlereagh,MULTIPOLYGON (((146.52897465087446 -32.3127849...,0.000703,0.435871,0.472666,0.090229,0.000531


In [40]:
ph.rename(columns = {"bin_0" : "ph_bin_0", "bin_1" : "ph_bin_1", "bin_2" : "ph_bin_2", "bin_3" : "ph_bin_3", "bin_4" : "ph_bin_4"}, inplace = True)
ph

,RegionName,geometry,ph_bin_0,ph_bin_1,ph_bin_2,ph_bin_3,ph_bin_4
0,QLD Condamine-Balonne,MULTIPOLYGON (((146.90174272390513 -28.9567308...,0.000009,0.304620,0.552931,0.140210,0.002229
1,NSW Barwon Darling,MULTIPOLYGON (((144.1874344326267 -29.03522301...,0.000000,0.091398,0.515253,0.354172,0.039177
2,QLD Moonie,MULTIPOLYGON (((148.6032494139603 -28.94575887...,0.000000,0.181288,0.742814,0.075898,0.000000
3,NSW Murray Above,MULTIPOLYGON (((147.31699138040665 -35.9670059...,0.003034,0.890713,0.106253,0.000000,0.000000
4,NSW Lower Darling,MULTIPOLYGON (((141.05399239322216 -33.2822541...,0.000000,0.000000,0.103675,0.866637,0.029689
5,QLD Border Rivers,MULTIPOLYGON (((149.00561064400677 -28.9283685...,0.000000,0.288485,0.593593,0.117810,0.000113
6,NSW Border Rivers,MULTIPOLYGON (((149.15253736606823 -28.8515927...,0.000425,0.512433,0.358522,0.128547,0.000072
7,NSW Gwydir,MULTIPOLYGON (((151.682735563879 -29.954786754...,0.000000,0.202680,0.449123,0.348197,0.000000
8,NSW Namoi,MULTIPOLYGON (((148.33494353037074 -29.9347375...,0.000000,0.241491,0.527929,0.229911,0.000669
9,NSW Macquarie-Castlereagh,MULTIPOLYGON (((146.52897465087446 -32.3127849...,0.000703,0.435871,0.472666,0.090229,0.000531


In [41]:
soil_frame = nitrogen.merge(phosphorus, on = ["RegionName", "geometry"], how = "inner")
soil_frame

,RegionName,geometry,n_bin_0,n_bin_1,n_bin_2,n_bin_3,n_bin_4,n_bin_5,n_bin_6,p_bin_0,p_bin_1,p_bin_2,p_bin_3,p_bin_4,p_bin_5,p_bin_6
0,QLD Condamine-Balonne,MULTIPOLYGON (((146.90174272390513 -28.9567308...,0.628219,0.347783,0.021059,0.002435,0.000456,0.000047,0.000000,0.933798,0.063448,2.478503e-03,0.000247,2.851467e-05,0.000000,0.000000
1,NSW Barwon Darling,MULTIPOLYGON (((144.1874344326267 -29.03522301...,0.973983,0.023493,0.002314,0.000208,0.000002,0.000000,0.000000,0.998382,0.001485,1.274587e-04,0.000005,9.306953e-08,0.000000,0.000000
2,QLD Moonie,MULTIPOLYGON (((148.6032494139603 -28.94575887...,0.742361,0.257634,0.000005,0.000000,0.000000,0.000000,0.000000,0.999994,0.000006,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000
3,NSW Murray Above,MULTIPOLYGON (((147.31699138040665 -35.9670059...,0.000000,0.006054,0.699257,0.213501,0.040172,0.040554,0.000462,0.000190,0.088860,3.516639e-01,0.415737,1.052755e-01,0.037934,0.000340
4,NSW Lower Darling,MULTIPOLYGON (((141.05399239322216 -33.2822541...,0.510316,0.487054,0.002530,0.000100,0.000000,0.000000,0.000000,0.909281,0.090719,8.133159e-07,0.000000,0.000000e+00,0.000000,0.000000
5,QLD Border Rivers,MULTIPOLYGON (((149.00561064400677 -28.9283685...,0.406046,0.539823,0.054054,0.000076,0.000000,0.000000,0.000000,0.959824,0.038245,1.727408e-03,0.000198,4.581984e-06,0.000000,0.000000
6,NSW Border Rivers,MULTIPOLYGON (((149.15253736606823 -28.8515927...,0.133807,0.675725,0.189032,0.001422,0.000014,0.000000,0.000000,0.678183,0.306903,1.446858e-02,0.000443,2.077088e-06,0.000000,0.000000
7,NSW Gwydir,MULTIPOLYGON (((151.682735563879 -29.954786754...,0.242226,0.667477,0.088964,0.001324,0.000009,0.000000,0.000000,0.683479,0.303165,1.320707e-02,0.000148,3.244901e-07,0.000000,0.000000
8,NSW Namoi,MULTIPOLYGON (((148.33494353037074 -29.9347375...,0.350251,0.593654,0.055366,0.000706,0.000017,0.000005,0.000001,0.758784,0.212159,2.533737e-02,0.003639,8.130264e-05,0.000000,0.000000
9,NSW Macquarie-Castlereagh,MULTIPOLYGON (((146.52897465087446 -32.3127849...,0.518498,0.451991,0.029445,0.000064,0.000002,0.000000,0.000000,0.837215,0.153014,8.491219e-03,0.001271,8.903545e-06,0.000000,0.000000


In [ ]:
soil_frame = soil_frame.merge(ph, on = ["RegionName", "geometry"], how = "inner")
soil_frame

In [43]:
soil_frame.drop("geometry", axis = 1, inplace = True)
soil_frame

,RegionName,n_bin_0,n_bin_1,n_bin_2,n_bin_3,n_bin_4,n_bin_5,n_bin_6,p_bin_0,p_bin_1,p_bin_2,p_bin_3,p_bin_4,p_bin_5,p_bin_6,ph_bin_0,ph_bin_1,ph_bin_2,ph_bin_3,ph_bin_4
0,QLD Condamine-Balonne,0.628219,0.347783,0.021059,0.002435,0.000456,0.000047,0.000000,0.933798,0.063448,2.478503e-03,0.000247,2.851467e-05,0.000000,0.000000,0.000009,0.304620,0.552931,0.140210,0.002229
1,NSW Barwon Darling,0.973983,0.023493,0.002314,0.000208,0.000002,0.000000,0.000000,0.998382,0.001485,1.274587e-04,0.000005,9.306953e-08,0.000000,0.000000,0.000000,0.091398,0.515253,0.354172,0.039177
2,QLD Moonie,0.742361,0.257634,0.000005,0.000000,0.000000,0.000000,0.000000,0.999994,0.000006,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.181288,0.742814,0.075898,0.000000
3,NSW Murray Above,0.000000,0.006054,0.699257,0.213501,0.040172,0.040554,0.000462,0.000190,0.088860,3.516639e-01,0.415737,1.052755e-01,0.037934,0.000340,0.003034,0.890713,0.106253,0.000000,0.000000
4,NSW Lower Darling,0.510316,0.487054,0.002530,0.000100,0.000000,0.000000,0.000000,0.909281,0.090719,8.133159e-07,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.103675,0.866637,0.029689
5,QLD Border Rivers,0.406046,0.539823,0.054054,0.000076,0.000000,0.000000,0.000000,0.959824,0.038245,1.727408e-03,0.000198,4.581984e-06,0.000000,0.000000,0.000000,0.288485,0.593593,0.117810,0.000113
6,NSW Border Rivers,0.133807,0.675725,0.189032,0.001422,0.000014,0.000000,0.000000,0.678183,0.306903,1.446858e-02,0.000443,2.077088e-06,0.000000,0.000000,0.000425,0.512433,0.358522,0.128547,0.000072
7,NSW Gwydir,0.242226,0.667477,0.088964,0.001324,0.000009,0.000000,0.000000,0.683479,0.303165,1.320707e-02,0.000148,3.244901e-07,0.000000,0.000000,0.000000,0.202680,0.449123,0.348197,0.000000
8,NSW Namoi,0.350251,0.593654,0.055366,0.000706,0.000017,0.000005,0.000001,0.758784,0.212159,2.533737e-02,0.003639,8.130264e-05,0.000000,0.000000,0.000000,0.241491,0.527929,0.229911,0.000669
9,NSW Macquarie-Castlereagh,0.518498,0.451991,0.029445,0.000064,0.000002,0.000000,0.000000,0.837215,0.153014,8.491219e-03,0.001271,8.903545e-06,0.000000,0.000000,0.000703,0.435871,0.472666,0.090229,0.000531


In [44]:
soil = pd.read_csv("soil_types_categories.csv")
soil

,RegionName,geometry,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,type_9,type_10,type_11,type_12,type_13
0,QLD Condamine-Balonne,MULTIPOLYGON (((146.90174272390513 -28.9567308...,0.353471,0.220491,0.056798,0.006227,0.003278,0.000038,0.033058,0.325764,5.787290e-05,0.000747,0.000070,0.000000e+00,0.000000
1,NSW Barwon Darling,MULTIPOLYGON (((144.1874344326267 -29.03522301...,0.342358,0.013995,0.004689,0.031734,0.000015,0.000000,0.011035,0.593589,0.000000e+00,0.002571,0.000013,0.000000e+00,0.000000
2,QLD Moonie,MULTIPOLYGON (((148.6032494139603 -28.94575887...,0.350449,0.292626,0.086244,0.000943,0.000000,0.000000,0.002139,0.267597,0.000000e+00,0.000002,0.000000,0.000000e+00,0.000000
3,NSW Murray Above,MULTIPOLYGON (((147.31699138040665 -35.9670059...,0.000000,0.000129,0.429435,0.196791,0.001963,0.004380,0.197636,0.168374,2.937375e-05,0.001234,0.000000,2.937375e-05,0.000000
4,NSW Lower Darling,MULTIPOLYGON (((141.05399239322216 -33.2822541...,0.120436,0.010569,0.035184,0.178730,0.000000,0.000000,0.021655,0.118324,0.000000e+00,0.001021,0.514081,0.000000e+00,0.000000
5,QLD Border Rivers,MULTIPOLYGON (((149.00561064400677 -28.9283685...,0.302172,0.528712,0.031920,0.025232,0.000000,0.005161,0.065984,0.027827,8.438534e-06,0.012983,0.000000,0.000000e+00,0.000000
6,NSW Border Rivers,MULTIPOLYGON (((149.15253736606823 -28.8515927...,0.342608,0.186863,0.088423,0.073331,0.002445,0.012773,0.168647,0.065390,3.148609e-05,0.059489,0.000000,4.142907e-07,0.000000
7,NSW Gwydir,MULTIPOLYGON (((151.682735563879 -29.954786754...,0.557178,0.070606,0.064176,0.146178,0.005733,0.005651,0.120695,0.014879,0.000000e+00,0.014903,0.000000,1.295201e-06,0.000000
8,NSW Namoi,MULTIPOLYGON (((148.33494353037074 -29.9347375...,0.411853,0.170025,0.053989,0.213425,0.001167,0.001881,0.135747,0.004156,7.786928e-07,0.007740,0.000011,0.000000e+00,0.000004
9,NSW Macquarie-Castlereagh,MULTIPOLYGON (((146.52897465087446 -32.3127849...,0.267084,0.016300,0.036248,0.457178,0.001873,0.001369,0.097652,0.118053,2.554723e-06,0.004230,0.000010,0.000000e+00,0.000000


In [45]:
soil.drop("geometry", inplace = True, axis = 1)

In [46]:
soil_frame = soil_frame.merge(soil, on = "RegionName", how = "inner")
soil_frame

,RegionName,n_bin_0,n_bin_1,n_bin_2,n_bin_3,n_bin_4,n_bin_5,n_bin_6,p_bin_0,p_bin_1,...,type_4,type_5,type_6,type_7,type_8,type_9,type_10,type_11,type_12,type_13
0,QLD Condamine-Balonne,0.628219,0.347783,0.021059,0.002435,0.000456,0.000047,0.000000,0.933798,0.063448,...,0.006227,0.003278,0.000038,0.033058,0.325764,5.787290e-05,0.000747,0.000070,0.000000e+00,0.000000
1,NSW Barwon Darling,0.973983,0.023493,0.002314,0.000208,0.000002,0.000000,0.000000,0.998382,0.001485,...,0.031734,0.000015,0.000000,0.011035,0.593589,0.000000e+00,0.002571,0.000013,0.000000e+00,0.000000
2,QLD Moonie,0.742361,0.257634,0.000005,0.000000,0.000000,0.000000,0.000000,0.999994,0.000006,...,0.000943,0.000000,0.000000,0.002139,0.267597,0.000000e+00,0.000002,0.000000,0.000000e+00,0.000000
3,NSW Murray Above,0.000000,0.006054,0.699257,0.213501,0.040172,0.040554,0.000462,0.000190,0.088860,...,0.196791,0.001963,0.004380,0.197636,0.168374,2.937375e-05,0.001234,0.000000,2.937375e-05,0.000000
4,NSW Lower Darling,0.510316,0.487054,0.002530,0.000100,0.000000,0.000000,0.000000,0.909281,0.090719,...,0.178730,0.000000,0.000000,0.021655,0.118324,0.000000e+00,0.001021,0.514081,0.000000e+00,0.000000
5,QLD Border Rivers,0.406046,0.539823,0.054054,0.000076,0.000000,0.000000,0.000000,0.959824,0.038245,...,0.025232,0.000000,0.005161,0.065984,0.027827,8.438534e-06,0.012983,0.000000,0.000000e+00,0.000000
6,NSW Border Rivers,0.133807,0.675725,0.189032,0.001422,0.000014,0.000000,0.000000,0.678183,0.306903,...,0.073331,0.002445,0.012773,0.168647,0.065390,3.148609e-05,0.059489,0.000000,4.142907e-07,0.000000
7,NSW Gwydir,0.242226,0.667477,0.088964,0.001324,0.000009,0.000000,0.000000,0.683479,0.303165,...,0.146178,0.005733,0.005651,0.120695,0.014879,0.000000e+00,0.014903,0.000000,1.295201e-06,0.000000
8,NSW Namoi,0.350251,0.593654,0.055366,0.000706,0.000017,0.000005,0.000001,0.758784,0.212159,...,0.213425,0.001167,0.001881,0.135747,0.004156,7.786928e-07,0.007740,0.000011,0.000000e+00,0.000004
9,NSW Macquarie-Castlereagh,0.518498,0.451991,0.029445,0.000064,0.000002,0.000000,0.000000,0.837215,0.153014,...,0.457178,0.001873,0.001369,0.097652,0.118053,2.554723e-06,0.004230,0.000010,0.000000e+00,0.000000


In [ ]:
corr_matrix = soil_frame.drop("RegionName", axis = 1).corr()

plt.figure(figsize = (12, 8))
sns.heatmap(corr_matrix, fmt = "0.2", cmap = "coolwarm", annot= True)
plt.show()

In [47]:
soil_frame.to_csv("soil_binned.csv", index = False)